<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/2.1-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

El objetivo de este notebook es enriquecer el dataset `user_reviews` con una nueva columna llamada `sentiment_analysis`. Esta columna contendrá el resultado de aplicar un análisis de sentimiento con NLP a las reseñas de los juegos escritas por los usuarios. De esta manera, podremos explorar la opinión de los usuarios sobre los diferentes juegos. Además, vamos a hacer un merge entre los distintos datasets para obtener una base de datos única y más eficiente para cada consulta, que nos permitirá optimizar el rendimiento de la API y reducir el tamaño de nuestra base de datos.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.

In [2]:
import sys
import os
import pandas as pd
import textblob
from textblob import TextBlob

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"TextBlob version: {textblob.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3
TextBlob version: 0.17.1


## 1 Extracción

En esta sección, extraemos los datos de los archivos `steam_games`, `user_items` y `user_reviews` que estan en formato parquet.

### 1.1 Extracción de los datos

Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.

In [3]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(f'{name}.parquet', engine='pyarrow')
    return dataframes

parquet_files = ['steam_games','user_items', 'user_reviews']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_steam_games = dataframes['steam_games']
df_user_items = dataframes['user_items']
df_user_reviews = dataframes['user_reviews']

## 2 Análisis de sentimiento

El análisis de sentimiento consiste en asignar una etiqueta numérica a cada reseña, según el tono o la actitud que expresa el texto. Usaremos la siguiente escala:

* 0: si la reseña es **negativa**, es decir, si el usuario muestra insatisfacción, disgusto o decepción con el juego.
* 1: si la reseña es **neutral**, es decir, si el usuario muestra indiferencia, objetividad o ausencia de emoción con el juego.
* 2: si la reseña es **positiva**, es decir, si el usuario muestra satisfacción, gusto o admiración con el juego.



### 2.1 Función `sentiment_analysis`

Para realizar el análisis de sentimiento con NLP a las reseñas de los juegos, crearemos una función usando la librería TextBlob que se considera facil de usar y muy intuitiva. Usaremos la polaridad que es una medida numérica que indica si el texto es negativo o positivo, según el tono o la actitud que expresa. La polaridad varía entre -1 y 1, donde -1 significa muy negativo, 0 significa neutro y 1 significa muy positivo.

In [4]:
def sentiment_analysis(review):
    # Si la reseña está ausente, retorna 1 (neutral)
    if pd.isnull(review):
        return 1

    # Calcula la polaridad de la reseña usando TextBlob
    polarity = TextBlob(review).sentiment.polarity

    # Retorna 0 (malo) si la polaridad es menor que 0, 2 (positivo) si la polaridad es mayor que 0, y 1 (neutral) en caso contrario
    if polarity < 0:
        return 0
    elif polarity > 0:
        return 2
    else:
        return 1

* Aplicamos la función a la columna `review`.

In [5]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(sentiment_analysis)

* Veamos algunos ejemplos:

In [6]:
df_user_reviews[['review','sentiment_analysis']].sample(5)

,review,sentiment_analysis
55192,Awesome game. Very hard especially in stealth ...,2
34001,it sucks,0
55241,"I don't own this game yet, I played it, once, ...",0
25366,"This game is the dog's ♥♥♥♥♥♥♥♥.Buy it, enjoy ...",1
36675,AMazing game 9/10 definetlythere is some bugs ...,0


### 2.2 Eliminación de la columna `review`

La nueva columna `sentiment_analysis` reemplazará a la columna `review` en el dataset `user_reviews`, para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [7]:
df_user_reviews.drop('review', axis=1, inplace=True)
df_user_reviews.head()

,item_id,recommend,user_id,posted_year,sentiment_analysis
0,1250,True,76561197970982479,2011,2
1,22200,True,76561197970982479,2011,2
2,43110,True,76561197970982479,2011,2
3,251610,True,js41637,2014,2
4,227300,True,js41637,2013,0


## 3 Diseño y estructura de las bases de datos para los endpoints de la API

En esta sección, nuestro objetivo es crear diferentes dataset a modo de pseudo base de datos para las funciones que se usarán en los endpoints de la API. De esta manera, podremos acceder a los datos que necesitamos de forma rápida y eficiente, sin tener que cargar toda la información para así, optimizar el rendimiento de la API.

### 3.1 Endpoints 1 y 2

Estos endpoints comparten información en común, por lo que podemos crear un solo dataset para ambos.

#### 3.1.1 Endpoint 1

def **PlayTimeGenre( *`genero` : str* )**:
    Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

``` js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```



#### 3.1.2 Endpoint 2

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:
```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```

#### 3.1.4 Pseudo Database 1

Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_items` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`genres`,`release_year` del DataFrame `steam_games`. También `item_id`, `user_id` y `playtime_forever` del DataFrame `user_items`.

* Primero, seleccionamos solo las columnas necesarias:

In [ ]:
steam_games_columns = ['item_id','genres','release_year']
user_items_columns = ['item_id','user_id', 'playtime_forever']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [ ]:
df_games_subset = df_steam_games[steam_games_columns]
df_items_subset = df_user_items[user_items_columns]

* Luego, hacemos un merge entre `steam_games` y `user_items` en la columna `item_id`.

In [ ]:
df_pseudo_db1 = pd.merge(df_games_subset, df_items_subset, on='item_id')
df_pseudo_db1.head()

,item_id,genres,release_year,user_id,playtime_forever
0,282010,Racing,1997,UTNerd24,0.083333
1,282010,Racing,1997,I_DID_911_JUST_SAYING,0.000000
2,282010,Racing,1997,76561197962104795,0.000000
3,282010,Racing,1997,r3ap3r78,0.000000
4,282010,Racing,1997,saint556,0.216667


In [ ]:
df_pseudo_db1.shape

(15255102, 5)

- Con el fin de ahorrar recursos, solo usaremos los registros de juegos que cumplan con las siguientes condiciones: tener un `release_year` válido, un `genres` con una popularidad alta tomando como referencia la frecuencia relativa y haber sido jugados al menos una vez.

In [ ]:
# Calculamos la frecuencia relativa de cada género
df_pseudo_db1.genres.value_counts()/len(df_pseudo_db1)

genres
Action                       0.229263
Adventure                    0.182320
Indie                        0.127814
Strategy                     0.098006
RPG                          0.087476
Simulation                   0.068177
Casual                       0.062629
Free to Play                 0.058383
Massively Multiplayer        0.051663
Racing                       0.010949
Sports                       0.008968
Early Access                 0.008278
Education                    0.004132
Utilities                    0.000933
Video Production             0.000321
Web Publishing               0.000259
Software Training            0.000170
unknown                      0.000129
Audio Production             0.000077
Photo Editing                0.000039
Animation &amp; Modeling     0.000009
Design &amp; Illustration    0.000005
Name: count, dtype: float64

- De acuerdo con la observación anterior, seleccionamos los 10 géneros más frecuentes, excluyendo las categorías que no consideramos un género como tal, sino que hacen referencia a por ejemplo si el juego es gratis (`Free to Play`) o tiene un acceso temprano (`Early Access`).

In [ ]:
# Seleccionamos los 10 géneros mas frecuentes
top_10_popular_genres = ['Action', 'Adventure', 'Indie', 'Strategy', 'RPG', 'Simulation', 'Casual', 'Massively Multiplayer', 'Racing', 'Sports']

# Filtramos por las condiciones establecidas
df_pseudo_db1 = df_pseudo_db1[(df_pseudo_db1['release_year'] != 'unknown') & (df_pseudo_db1['playtime_forever'] > 0) & (df_pseudo_db1['genres'].isin(top_10_popular_genres))].reset_index(drop=True)
df_pseudo_db1.shape

(9753088, 5)

- Para optimizar el uso de la memoria en el deploy, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [ ]:
df_pseudo_db1['release_year'] = df_pseudo_db1['release_year'].astype('int16')
df_pseudo_db1['playtime_forever'] = df_pseudo_db1['playtime_forever'].astype('float32')
df_pseudo_db1.memory_usage(deep=True)

Index                     132
item_id             609652065
genres              629485325
release_year         19506176
user_id             686351247
playtime_forever     39012352
dtype: int64

* Por último, creamos una tabla pivote que tenga como índice `user_id` y `release_year`, como columnas `genres` y como valores únicos la suma de `playtime_forever`.

In [ ]:
df_pseudo_db1 = df_pseudo_db1.pivot_table(index=['user_id', 'release_year'], columns='genres', values='playtime_forever', aggfunc='sum', fill_value=0)
df_pseudo_db1

genres                        Action   Adventure     Casual      Indie  \
user_id    release_year                                                  
--000--    2006            15.416667   15.416667   0.000000  15.416667   
           2009            88.816666   88.816666   0.000000   0.000000   
           2010             0.366667    0.000000   0.000000   0.366667   
           2011           108.699997  108.699997   0.000000  30.616665   
           2012          1822.516724   37.150002  30.016666  37.700001   
...                              ...         ...        ...        ...   
zzzmidmiss 2010             7.783334    0.166667   3.916667   7.950000   
           2011            38.366665   38.366665   1.250000   1.750000   
           2012            98.366669   61.650005   6.083333  51.316666   
           2013             1.633333    1.750000   0.283333   1.750000   
           2014             0.266667    0.266667   0.266667   0.000000   

genres                   Massively Multiplayer        RPG     Racing  \
user_id    release_year                                                
--000--    2006                      15.416667   0.000000   0.000000   
           2009                       0.000000   0.000000   0.000000   
           2010                       0.000000   0.000000   0.000000   
           2011                      46.049999  62.649998  46.049999   
           2012                      10.500000  29.516666   0.000000   
...                                        ...        ...        ...   
zzzmidmiss 2010                       0.000000   0.000000   0.000000   
           2011                       0.266667  37.599998   0.266667   
           2012                       8.016666  45.500000   0.000000   
           2013                       0.166667   0.166667   0.000000   
           2014                       0.266667   0.000000   0.000000   

genres                   Simulation    Sports     Strategy  
user_id    release_year                                     
--000--    2006           15.416667  0.000000     0.000000  
           2009            0.000000  0.000000     0.000000  
           2010            0.000000  0.000000     0.000000  
           2011           11.083333  0.000000    11.083333  
           2012            0.000000  7.683333  1796.400024  
...                             ...       ...          ...  
zzzmidmiss 2010            3.233333  3.233333     3.400000  
           2011            0.000000  0.000000     1.150000  
           2012            6.450000  0.000000    15.383334  
           2013            0.000000  0.000000     1.466667  
           2014            0.266667  0.266667     0.266667  

[660003 rows x 10 columns]

In [ ]:
df_pseudo_db1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 660003 entries, ('--000--', 2006) to ('zzzmidmiss', 2014)
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Action                 660003 non-null  float32
 1   Adventure              660003 non-null  float32
 2   Casual                 660003 non-null  float32
 3   Indie                  660003 non-null  float32
 4   Massively Multiplayer  660003 non-null  float32
 5   RPG                    660003 non-null  float32
 6   Racing                 660003 non-null  float32
 7   Simulation             660003 non-null  float32
 8   Sports                 660003 non-null  float32
 9   Strategy               660003 non-null  float32
dtypes: float32(10)
memory usage: 28.8+ MB


### 3.2 Endpoints 3, 4 y 5.

#### 3.2.1 Endpoint 3

+ def **UsersRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  

Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```


#### 3.2.2 Endpoint 4

+ def **UsersWorstDeveloper( *`año` : int* )**:
   Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
Ejemplo de retorno:
```js
[
   {
      "Puesto 1":"X"
   },
   {
      "Puesto 2":"Y"
   },
   {
      "Puesto 3":"Z"
   }
]
```

#### 3.2.3 Endpoint 5

def **sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:
```js
{
   "Valve":[
      Negative = 182,
      Neutral = 120,
      Positive = 278
   ]
}
```


#### 3.1.4 Pseudo Database 2

- Para crear un solo dataset que pueda ser utilizado como pseudo base de datos para estos endpoints, necesitamos combinar `df_steam_games` con `df_user_reviews` de tal manera que tengamos toda la información necesaria en un solo lugar. Para esto solo necesitamos las columnas:
`item_id`,`item_name`,`developer` del DataFrame `steam_games`. También `item_id`, `recommend`, `sentiment_analysis` y `posted_year` del DataFrame `user_reviews`.

- Primero, seleccionamos las columnas necesarias:

In [ ]:
steam_games_columns = ['item_id', 'item_name', 'developer']
user_reviews_columns = ['item_id', 'recommend','sentiment_analysis','posted_year']

* Segundo, creamos subsets de los DataFrames con solo las columnas necesarias:

In [ ]:
df_games_subset = df_steam_games[steam_games_columns]
df_reviews_subset = df_user_reviews[user_reviews_columns]

* Luego, hacemos un merge entre los subsets `steam_games` y `user_reviews` en la columna `item_id`.

In [ ]:
df_pseudo_db2 = pd.merge(df_games_subset, df_reviews_subset, on='item_id')
df_pseudo_db2.head()

,item_id,item_name,developer,recommend,sentiment_analysis,posted_year
0,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
1,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
2,282010,Carmageddon Max Pack,Stainless Games Ltd,True,1,unknown
3,70,Half-Life,Valve,True,0,2015
4,70,Half-Life,Valve,True,0,2011


- Con el objetivo de ahorrar recursos, filtraremos los registros de juegos que cumplan con las siguientes condiciones: tener un review con un `posted_year` válido y un `developer` conocido. Además, eliminaremos la columna `item_id`, ya que no la necesitamos.

In [ ]:
df_pseudo_db2 = df_pseudo_db2[(df_pseudo_db2['posted_year'] != 'unknown') & (df_pseudo_db2['developer'] != 'unknown') ].reset_index(drop=True)
df_pseudo_db2.drop('item_id',axis=1, inplace=True)
df_pseudo_db2.head()

,item_name,developer,recommend,sentiment_analysis,posted_year
0,Half-Life,Valve,True,0,2015
1,Half-Life,Valve,True,0,2011
2,Half-Life,Valve,True,0,2014
3,Half-Life,Valve,True,2,2013
4,Half-Life,Valve,True,0,2013


- Por último, para optimizar el uso de la memoria en el deploy de la API, convertiremos las columnas a los tipos de datos adecuados según su contenido.

In [ ]:
df_pseudo_db2['sentiment_analysis'] = df_pseudo_db2['sentiment_analysis'].astype('int8')
df_pseudo_db2['posted_year'] = df_pseudo_db2['posted_year'].astype('int16')
df_pseudo_db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152619 entries, 0 to 152618
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   item_name           152619 non-null  object
 1   developer           152619 non-null  object
 2   recommend           152619 non-null  bool  
 3   sentiment_analysis  152619 non-null  int8  
 4   posted_year         152619 non-null  int16 
dtypes: bool(1), int16(1), int8(1), object(2)
memory usage: 2.9+ MB


## 4 Feature Engineering for RecSys

Mejorar el rendimiento predictivo de un modelo
Reducir las necesidades computacionales o de datos
Mejorar la interpretabilidad de los resultados

Determinar qué características son las más importantes con información mutua
Inventar nuevas características en varios dominios de problemas del mundo real
Codifique categóricos de alta cardinalidad con una codificación de destino
Creación de entidades de segmentación con la agrupación en clústeres K-means
Descomponer la variación de un conjunto de datos en entidades con el análisis de componentes principales

* Primero, vamos unir los tres conjuntos de datos por el campo user_id, que es el identificador único de cada usuario. Esto nos permitirá tener un solo conjunto de datos con toda la información relevante sobre los usuarios, como los juegos que han comprado o jugado, las reseñas que han escrito, el análisis de sentimiento, etc. Vamos a usar la función merge de pandas para hacer esta operación.

In [8]:
df_user_games = pd.merge(df_user_items, df_steam_games, on='item_id')
df_user_games

,item_id,playtime_forever,user_id,publisher,item_name,price,early_access,developer,release_year,genres
0,10,0.100000,76561197970982479,Valve,Counter-Strike,9.99,False,Valve,2000,Action
1,10,0.100000,76561197970982479,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy
2,10,0.000000,js41637,Valve,Counter-Strike,9.99,False,Valve,2000,Action
3,10,0.000000,js41637,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy
4,10,0.000000,Riot-Punch,Valve,Counter-Strike,9.99,False,Valve,2000,Action
...,...,...,...,...,...,...,...,...,...,...
15255097,433920,0.000000,inven,Degica,Aveyond 4: Shadow of the Mist,14.99,False,"Amaranth Games, LLC",2016,RPG
15255098,433920,0.000000,inven,Degica,Aveyond 4: Shadow of the Mist,14.99,False,"Amaranth Games, LLC",2016,Indie
15255099,433920,0.000000,inven,Degica,Aveyond 4: Shadow of the Mist,14.99,False,"Amaranth Games, LLC",2016,Adventure
15255100,485270,0.116667,danebuchanan,unknown,Arachnophobia,4.99,False,unknown,unknown,Casual


In [9]:
df_final = pd.merge(df_user_games, df_user_reviews, on=['user_id','item_id'])
df_final.head()

,item_id,playtime_forever,user_id,publisher,item_name,price,early_access,developer,release_year,genres,recommend,posted_year,sentiment_analysis
0,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2015,0
1,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy,True,2015,0
2,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2011,2
3,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy,True,2011,2
4,10,168.550000,mayshowganmore,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2014,1


In [10]:
# Convertir las columnas de tipo object a tipo category
category_columns = df_final.drop(['price', 'playtime_forever'], axis=1).columns.tolist()
df_final[category_columns] = df_final[category_columns].astype('category')


In [25]:
type(category_columns)

list

In [11]:
df_features = df_final.copy()
df_features

,item_id,playtime_forever,user_id,publisher,item_name,price,early_access,developer,release_year,genres,recommend,posted_year,sentiment_analysis
0,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2015,0
1,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy,True,2015,0
2,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2011,2
3,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,Strategy,True,2011,2
4,10,168.550000,mayshowganmore,Valve,Counter-Strike,9.99,False,Valve,2000,Action,True,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155134,367010,1.733333,steviescoop,TripleBrick,Vernon's Legacy,11.99,False,TripleBrick,2016,Indie,True,unknown,0
155135,367010,1.733333,steviescoop,TripleBrick,Vernon's Legacy,11.99,False,TripleBrick,2016,Adventure,True,unknown,0
155136,381850,0.800000,kazanmor,Aerosoft GmbH,My Paper Boat,4.99,False,TML-Studios,2015,Adventure,True,2015,2
155137,381850,0.800000,kazanmor,Aerosoft GmbH,My Paper Boat,4.99,False,TML-Studios,2015,Casual,True,2015,2


In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155139 entries, 0 to 155138
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   item_id             155139 non-null  category
 1   playtime_forever    155139 non-null  float64 
 2   user_id             155139 non-null  category
 3   publisher           155139 non-null  category
 4   item_name           155139 non-null  category
 5   price               155139 non-null  float64 
 6   early_access        155139 non-null  category
 7   developer           155139 non-null  category
 8   release_year        155139 non-null  category
 9   genres              155139 non-null  category
 10  recommend           155139 non-null  category
 11  posted_year         155139 non-null  category
 12  sentiment_analysis  155139 non-null  category
dtypes: category(11), float64(2)
memory usage: 6.9 MB


In [13]:
!pip install category_encoders

In [16]:
# Crear una lista con el valor 'desconocido'
lista_desconocido = ['unknown']

# Filtrar el conjunto de datos para obtener solo los registros con desconocido
df_desconocido = df_final.isin(lista_desconocido)

# Ver el resultado
df_desconocido.loc[True]


KeyError: ignored

In [22]:
# Importar la librería
import category_encoders as ce

# Crear un codificador de un solo bit para las columnas categóricas
encoder = ce.OneHotEncoder(cols=['recommend', 'sentiment_analysis', 'genres'])

# Aplicar el codificador al conjunto de datos
df_codificado = encoder.fit_transform(df_features)


In [23]:
df_codificado

,item_id,playtime_forever,user_id,publisher,item_name,price,early_access,developer,release_year,genres_1,...,genres_16,genres_17,genres_18,genres_19,recommend_1,recommend_2,posted_year,sentiment_analysis_1,sentiment_analysis_2,sentiment_analysis_3
0,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,1,...,0,0,0,0,1,0,2015,1,0,0
1,10,405.600000,Bennysaputra,Valve,Counter-Strike,9.99,False,Valve,2000,0,...,0,0,0,0,1,0,2015,1,0,0
2,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,1,...,0,0,0,0,1,0,2011,0,1,0
3,10,64.916667,76561198040188061,Valve,Counter-Strike,9.99,False,Valve,2000,0,...,0,0,0,0,1,0,2011,0,1,0
4,10,168.550000,mayshowganmore,Valve,Counter-Strike,9.99,False,Valve,2000,1,...,0,0,0,0,1,0,2014,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155134,367010,1.733333,steviescoop,TripleBrick,Vernon's Legacy,11.99,False,TripleBrick,2016,0,...,0,0,0,0,1,0,unknown,1,0,0
155135,367010,1.733333,steviescoop,TripleBrick,Vernon's Legacy,11.99,False,TripleBrick,2016,0,...,0,0,0,0,1,0,unknown,1,0,0
155136,381850,0.800000,kazanmor,Aerosoft GmbH,My Paper Boat,4.99,False,TML-Studios,2015,0,...,0,0,0,0,1,0,2015,0,1,0
155137,381850,0.800000,kazanmor,Aerosoft GmbH,My Paper Boat,4.99,False,TML-Studios,2015,0,...,0,0,0,0,1,0,2015,0,1,0


In [24]:
df_codificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155139 entries, 0 to 155138
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   item_id               155139 non-null  category
 1   playtime_forever      155139 non-null  float64 
 2   user_id               155139 non-null  category
 3   publisher             155139 non-null  category
 4   item_name             155139 non-null  category
 5   price                 155139 non-null  float64 
 6   early_access          155139 non-null  category
 7   developer             155139 non-null  category
 8   release_year          155139 non-null  category
 9   genres_1              155139 non-null  int64   
 10  genres_2              155139 non-null  int64   
 11  genres_3              155139 non-null  int64   
 12  genres_4              155139 non-null  int64   
 13  genres_5              155139 non-null  int64   
 14  genres_6              155139 non-nul

In [ ]:
df_user_items.columns

Index(['item_id', 'playtime_forever', 'user_id'], dtype='object')

In [ ]:
df_user_reviews.columns

Index(['item_id', 'recommend', 'user_id', 'posted_year', 'sentiment_analysis'], dtype='object')

In [ ]:
df_steam_games.columns

Index(['publisher', 'item_name', 'price', 'early_access', 'item_id',
       'developer', 'release_year', 'genres'],
      dtype='object')

## 5 Carga

Finalmente, en esta sección cargamos nuestros datos transformados para los endpoints que se consumirán en la API a su destino final. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [ ]:
# Nombres correspondientes a cada DataFrame
dfs = [df_pseudo_db1, df_pseudo_db2, df_user_reviews]
names = ['pseudo-db1.parquet', 'pseudo-db2.parquet', 'user_sentiment_analysis.parquet' ]

for dfs, n in zip(dfs, names):
    # Definimos la ruta del directorio
    folder_path = f'../data/processed/'

    # Verificamos si el folder_path existe
    if not os.path.exists(folder_path):
        # Si no existe, lo creamos
        os.makedirs(folder_path)

    # Definimos la ruta completa del archivo
    path = os.path.join(folder_path, n)

    # Guardamos el DataFrame como un archivo parquet
    dfs.to_parquet(path, engine='pyarrow', compression='snappy')

    print(f"'{n}' fue guardado correctamente en '{folder_path}'")

'pseudo-db1.parquet' fue guardado correctamente en '../data/processed/'
'pseudo-db2.parquet' fue guardado correctamente en '../data/processed/'
'user_sentiment_analysis.parquet' fue guardado correctamente en '../data/processed/'
